In [ ]:
# import required libraries
import pandas as pd
import pickle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from nltk.stem.porter import PorterStemmer

#required function definitions

stemmer = PorterStemmer()

def stem_words(text):
   return " ".join([stemmer.stem(word) for word in text.split()])

STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Load the test data
!gdown --id 1F-7nLUlorJJxDUr0Ua_wNdQPRlkN5dLe
test_df=pd.read_csv('/content/test data (pre-inter iit).csv')

# Load the trained model
!gdown --id 19E4etZq_D1S_HlfQzNssp3zaO86eicrx
!gdown --id 1gPdyTpgI6PXx-mIJ-kCXUkdWvBjORVhU
clf = pickle.load(open('/content/finalized_model.sav', 'rb'))
cv = pickle.load(open('/content/TfidfVectorizer.sav', 'rb'))

Downloading...
From: https://drive.google.com/uc?id=1F-7nLUlorJJxDUr0Ua_wNdQPRlkN5dLe
To: /content/test data (pre-inter iit).csv
100% 7.67M/7.67M [00:00<00:00, 29.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=19E4etZq_D1S_HlfQzNssp3zaO86eicrx
To: /content/finalized_model.sav
100% 309M/309M [00:07<00:00, 42.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gPdyTpgI6PXx-mIJ-kCXUkdWvBjORVhU
To: /content/TfidfVectorizer.sav
100% 5.49M/5.49M [00:00<00:00, 20.8MB/s]


In [ ]:
%%time

test_df['X'] = test_df['title']+' '+test_df['title']+' '+test_df['content']
test_df['X'] = test_df['X'].apply(clean_text)
test_df['title'].replace( { r'[^a-z ]' : '' }, inplace= True, regex = True)

X_test = test_df["X"].apply(lambda text: stem_words(text))

X_test_cv = cv.transform(X_test)

preds=clf.predict(X_test_cv)

**NOTE:**
*   Time of inferance when GPU is used: 22.5secs
*   Time of inferance when CPU is used: 23.2 secs

In [ ]:
# Save Submission File
test_df.drop('title',axis=1,inplace=True)
test_df.drop('content',axis=1,inplace=True)
test_df.drop('X',axis=1,inplace=True)
test_df['target_ind']=preds
test_df.to_csv('submission.csv',index=False)